Making dataframe from a wikipeia page.
Columns of the dataframe are **PostalCode**, **Borough**, and **Neighborhood**.

In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df.columns = ['PostalCode','Borough','Neighbourhood']
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Eliminate rows where a **borough** is **Not assgined**.

In [2]:
df2=df[df.Borough != 'Not assigned']
print(df2.shape)
df2.head()

(210, 3)


,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Combining multiple rows which have same **PostalCode** and **Borough**.

In [3]:
df3= df2.groupby(['PostalCode','Borough'],as_index=False).agg(lambda x: ','.join(x))
df3.reset_index()
print(df3.shape)
df3.head()

(103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


For cells whose **Neighbourhood** is **Not assigned**, the **Neighbourhood** will be the same as **Borough**. 

In [4]:
df3.loc[df3['Neighbourhood']=="Not assigned",'Neighbourhood']=df3.loc[df3['Neighbourhood']=="Not assigned",'Borough']

print('The shape of dataframe is ',df3.shape)
df3.head()

The shape of dataframe is  (103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Get the geographical coordinates of each postal code from csv file.

In [5]:
#!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data

df_geo = pd.read_csv("Geospatial_Coordinates.csv")
df_geo.columns = ["PostalCode","Latitude","Longitude"]
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging **Postal Code dataframe** and **Geographical coordinates** dataframe.

In [6]:
df_merged = pd.merge(df3,df_geo)
print(df_merged.shape)
display(df_merged)

(103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


Because **geopy** is not stable, I will use the mean of each neighbour's Latitude and Longitude as the center of the map.

In [7]:
latitude = df_merged['Latitude'].mean()
longitude = df_merged['Longitude'].mean()

print('The cender of Toronto are {}, {}.'.format(latitude, longitude))

The cender of Toronto are 43.70460773398059, -79.39715291165048.


In [8]:
import folium

# create map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto